In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# Download the data for January and February 2023
url_january = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'
url_february = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'

In [3]:
# Read the January dataset from a Parquet file
df_january = pd.read_parquet(url_january)

# Read the February dataset from a Parquet file
df_february = pd.read_parquet(url_february)

In [4]:
# Show dataframe
df_january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
# Show dataframe
df_february.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [6]:
# Question 1: How many columns does the January dataset have?

# Get the number of columns in the January DataFrame
num_columns_january = df_january.shape[1]
print("The number of columns in the January DataFrame is:", num_columns_january)

The number of columns in the January DataFrame is: 19


In [7]:
# Question 2: Compute the duration variable in minutes

# Convert 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' columns to datetime
df_january['tpep_pickup_datetime'] = pd.to_datetime(df_january['tpep_pickup_datetime'])
df_january['tpep_dropoff_datetime'] = pd.to_datetime(df_january['tpep_dropoff_datetime'])

# Calculate the duration of each trip in minutes and store it in a new column 'duration'
df_january['duration'] = (df_january['tpep_dropoff_datetime'] - df_january['tpep_pickup_datetime']).dt.total_seconds() / 60

# Calculate the standard deviation of travel times in January
std_typical_deviation = df_january['duration'].std()
print("The standard deviation of January travel times is:", std_typical_deviation, "minutes.")

The standard deviation of January travel times is: 42.594351241920904 minutes.


In [8]:
# Question 3: Drop outliers

# Filter the January DataFrame to include only trips with duration between 1 and 60 minutes
df_filtered = df_january[(df_january['duration'] >= 1) & (df_january['duration'] <= 60)]

# Calculate the fraction of records remaining after removing outliers
fraction_records = len(df_filtered) / len(df_january)
print("The fraction of records after removing outliers is:", fraction_records)

The fraction of records after removing outliers is: 0.9812202822125979


In [9]:
# Question 4: Apply one-hot encoding
# Define the categorical and numerical columns
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# Copy the filtered DataFrame
df_filtered_copy = df_filtered.copy()

# Convert categorical columns to str type in the DataFrame copy.
df_filtered_copy[categorical] = df_filtered_copy[categorical].astype(str)

# Now modify the columns in the copy of the DataFrame using .loc[].
df_filtered_copy.loc[:, categorical] = df_filtered_copy[categorical].astype(str)

# Defines the characteristics
features = categorical + numerical

# Create training dictionaries
train_dicts = df_filtered_copy[features].to_dict(orient='records')

In [10]:
# Initialize a DictVectorizer object
dv = DictVectorizer()

# Fit and transform the training dictionaries to create the feature matrix
X_train = dv.fit_transform(train_dicts)

# Get the number of columns in the feature matrix
num_columns = len(dv.feature_names_)
print("The number of columns in the matrix is:", num_columns)

The number of columns in the matrix is: 516


In [11]:
# Question 5: Train the model

# Extract the target variable 'duration' from the filtered DataFrame and convert it to a NumPy array
y_train = df_filtered['duration'].values

In [12]:
# Initialize a LinearRegression model
lr = LinearRegression()

# Fit the model to the training data
lr.fit(X_train, y_train)

# Make predictions on the training data
y_pred = lr.predict(X_train)

# Calculate the root mean squared error (RMSE) on the training data
rmse_train = mean_squared_error(y_train, y_pred, squared=False)
print("The RMSE on train is:", rmse_train)

The RMSE on train is: 7.658406323428972


In [13]:
# Evaluate the model on the validation dataset (February 2023)

# Convert 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' columns to datetime
df_february['tpep_pickup_datetime'] = pd.to_datetime(df_february['tpep_pickup_datetime'])
df_february['tpep_dropoff_datetime'] = pd.to_datetime(df_february['tpep_dropoff_datetime'])

# Calculate the duration of each trip in minutes and store it in a new column 'duration'
df_february['duration'] = (df_february['tpep_dropoff_datetime'] - df_february['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter the data to include only trips with a duration between 1 and 60 minutes
df_february_filtered = df_february[(df_february['duration'] >= 1) & (df_february['duration'] <= 60)]

# Define the categorical and numerical columns
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# Create a copy of the filtered DataFrame
df_february_filtered_copy = df_february_filtered.copy()

# Convert categorical columns to type 'str' in the DataFrame copy
df_february_filtered_copy[categorical] = df_february_filtered_copy[categorical].astype(str)

# Modify the columns in the DataFrame copy using .loc[]
df_february_filtered_copy.loc[:, categorical] = df_february_filtered_copy[categorical].astype(str)

In [14]:
# Create validation dictionaries from the filtered February DataFrame
val_dicts = df_february_filtered_copy[features].to_dict(orient='records')

# Transform the validation data using the same DictVectorizer object used for training
X_val = dv.transform(val_dicts)

# Extract the target variable 'duration' from the filtered February DataFrame and convert it to a NumPy array
y_val = df_february_filtered['duration'].values

In [15]:
# Make predictions on the validation data
y_val_pred = lr.predict(X_val)

# Calculate the root mean squared error (RMSE) on the validation data
rmse_val = mean_squared_error(y_val, y_val_pred, squared=False)
print("The RMSE on validation is:", rmse_val)

The RMSE on validation is: 7.820110499730046
